In [38]:
import pandas as pd
import numpy as np
from duckduckgo_search import DDGS
import json
import html2text
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
import os

def ddg_search(input_text,max_results=None):
    """
    parameter: input_text is a keyword to search
    The function fetch all the url related to given text and saves in json format
    """
    file_path = "ddgs.json"
    if os.path.exists(file_path):
        # Delete the file
        os.remove(file_path)
        print("File deleted successfully.")
    else:
        print("File does not exist.")

    with DDGS() as ddgs:
        results = [r for r in ddgs.text(f"{input_text}", safesearch = "off",timelimit = "d",max_results = max_results)]
    
    # Write list of dictionaries to JSON file
    with open(file_path, "w") as json_file:
        json.dump(results, json_file)
        print("file created successfully")



def do_webscraping(json_file):
    """
    parameter: it takes json file
    returns a list of dictionaries including page content, title, metadata and clean text of the urls
    """
    with open(f"{json_file}", "r") as json_file:
        data_list = json.load(json_file)

    urls = [x['href'] for x in data_list]

    structured_response = []
    for url in urls:
        try:
            loader = AsyncHtmlLoader(url)
            docs = loader.load()

            html2text_transformer = Html2TextTransformer()
            docs_transformed = html2text_transformer.transform_documents(docs)

            if docs_transformed is not None and len(docs_transformed) > 0:
                metadata = docs_transformed[0].metadata
                title = metadata.get('title', '')

                structured_response.append({
                    'summary': docs_transformed[0].page_content,
                    'title': title,
                    'metadata': metadata,
                    'clean_content': html2text.html2text(docs_transformed[0].page_content)
                })
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            continue
    
    return structured_response


In [39]:
input_text = "tata technology stocks"

In [40]:
ddg_search(input_text,max_results=5)

File deleted successfully.
file created successfully


In [41]:

response = do_webscraping(json_file="ddgs.json")


Fetching pages:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.78it/s]


In [56]:
text = response[0]['clean_content']

Create Embeddings

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain


In [44]:
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [45]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

In [48]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [63]:
def get_sentiment():
    prompt_template = """
            f'This is a sentiment analysis request. 
            What is the sentiment of the following text, and why?\n\n"{context}"\n\nSentiment: {{sentiment}}')

    """
    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.9)


    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

In [59]:
                raw_text = get_pdf_text(pdf_docs)
                text_chunks = get_text_chunks(raw_text)
                get_vector_store(text_chunks)

ValueError: Unexpected message with type <class 'str'> at the position 0.